In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [73]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def create_agent(llm, tools, system_message: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)


In [74]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


from langgraph.prebuilt import ToolNode

available_tools = [get_word_length]
tools_node = ToolNode(available_tools)

In [75]:
from langchain_openai import ChatOpenAI
client_openai = ChatOpenAI(temperature=0)
client_openai.bind_tools(available_tools)

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000022659D20A10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000022659D21280>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_word_length', 'description': 'Returns the length of a word.', 'parameters': {'type': 'object', 'properties': {'word': {'type': 'string'}}, 'required': ['word']}}}]})

In [76]:
agent1 = create_agent(client_openai, [get_word_length], "")


In [77]:
from langgraph.graph import END, StateGraph, MessagesState

def condition_routing(state: MessagesState):
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state['messages']
    # response = agent1.invoke(messages[-2:])
    response = agent1.invoke(messages)
    return {"messages": [response]}


In [78]:

from langgraph.checkpoint import MemorySaver


workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools",tools_node)
workflow.add_conditional_edges("agent", condition_routing)
workflow.add_edge("tools", "agent")
workflow.set_entry_point("agent")

# memory = MemorySaver()
# app = workflow.compile(checkpointer=memory)
app = workflow.compile()


In [79]:
final_state = app.invoke(dict(messages=[HumanMessage(content="how many letters in w")]))
final_state["messages"][-1].content

IndexError: list index out of range